In [ ]:
import geopandas as gpd
import rasterio
import os
from glob import glob
from natsort import natsorted
from rasterio.mask import mask
import numpy as np
from tqdm import tqdm
import pandas as pd

In [ ]:
gpkg_path = "input_data.gpkg"
gdf = gpd.read_file(gpkg_path)
gdf.columns
### Index(['PLOT_NO', 'label',  'geometry'])

### downloaded image large area with splited part(row_col)

"B2", "B3", "B4", "B8" timeseries data

In [ ]:
### extract date list from one sigle folder
raster_path = natsorted(glob("/B2/row0_col0/*.tif"))[:-1]
raster_path
date_list = []
for file in raster_path:
    date_list.append(os.path.basename(file)[:8])
date_list

In [ ]:
grand_list = []
for idx, row in tqdm(gdf.iterrows(), total=len(gdf)):
    geom = row.geometry
    label = row.get('label')
    PLOT_NO = row.get('PLOT_NO')
    df_list = []
    time_stack_l = []
    for d in date_list:
        band_stack_l = []
        for band in ["B2", "B3", "B4", "B8"]:
            file = f"/raster_files/{band}/{d}_{band}.tif"
            with rasterio.open(file) as src:
                masked_image, _ = mask(src, [geom], crop=True)
                band_stack_l.append(masked_image)
        band_stack = np.concatenate(band_stack_l, axis=0)
        time_stack_l.append(band_stack)
    time_stack = np.stack(time_stack_l, axis=0)
    T, F, H, W = time_stack.shape
    # Conver to time, feature and pixel
    reshaped_time_stack = time_stack.reshape(T, F, H*W)
    # Remove nan values from per temporal pixel
    for i in range(reshaped_time_stack.shape[-1]):
        if np.isnan(reshaped_time_stack[:,:, i]).any():
            continue
        df_list.append(
        [reshaped_time_stack[:,:,i], label]
    )
    df = pd.DataFrame(df_list)
    grand_list.append(df)
grand_df = pd.concat(grand_list)
grand_df.to_csv("training_data.csv", index=False)